# Tutorial 4: Improving LLMs with RLHF

Reinforcement Learning from Human Feedback (RLHF) incorporates human feedback into the training process through a reward model that learns the desired patterns to improve the model’s output. For example, if the goal is to enhance politeness, the reward model will guide the model to generate more polite responses by assigning higher scores to polite outputs. This process is resource-intensive because it necessitates training a reward model using a dataset curated by humans.

This tutorial will use available open-source models and datasets whenever possible while maintaining costs.

We begin with a pre-trained model that we fine-tune in a supervised fine-tuning phase using the `SFTTrainer` class. Next, a reward model is trained with the desired traits using the `RewardTrainer` class. Finally, the reinforcement learning phase employs the models to build the ultimate aligned model, utilizing the `PPOTrainer`.

You can access the reports generated from Weights & Biases and the file with the requirements for the library after each subsection. Note that different steps require distinct versions of libraries. We chose `OPT-1.3B` as the base model and fine-tuned a `DeBERTa` (300M) model as the reward model for our experiments. While these are more compact models, the process used in this tutorial can be applied to other existing models by simply modifying the model’s name in the code.

Even if much more affordable than what companies like OpenAI do, this tutorial is still resource-intensive as we replicate an RLHF phase. We rented an 8x NVIDIA A100 instance for $8.80/h and used  [lambda](https://lambdalabs.com/)  as our GPU cloud provider.

>⚠️It’s important to be aware of the costs associated with cloud GPUs. The total cost will depend on the machine type and the instance’s uptime. Regularly check your costs in the billing section of Lambda Labs and spin off your instances when you don’t use them.

>💡If you want to run the code in the section without spending much money, you can perform a few iterations of training on your virtual machine and then stop it.

## Supervised Fine-Tuning

• Find the  [Notebook](https://colab.research.google.com/github/towardsai/ragbook-notebooks/blob/main/notebooks/Chapter%2010%20-%20FineTuning_a_LLM_QLoRA.ipynb)  for this section at  [towardsai.net/book](http://towardsai.net/book).

Previous sections covered the SFT phase. This section uses a unique  [OpenOrca](https://huggingface.co/datasets/Open-Orca/OpenOrca)  dataset with question-response pairs and implements the QLoRA fine-tuning technique.

This phase teaches the model a conversational format, training it to provide answers rather than defaulting to its standard auto-completion function.

Install the required libraries with the command `!pip install -q transformers==4.32.0 bitsandbytes==0.41.1 accelerate==0.22.0 deeplake==3.6.19 trl==0.5.0 peft==0.5.0 wandb==0.15.8.`

### The Dataset

The first step is streaming the dataset. Streaming a dataset refers to the process of loading and processing data in smaller chunks or batches rather than loading the entire dataset into memory at once. This approach is useful when working with large datasets that cannot fit into memory or when you want to perform real-time processing. For this example, we only use a subset of the original dataset, comprising 1 million data points. However, you can access the  [entire dataset](https://app.activeloop.ai/genai360/OpenOrca-4M/), containing 4 million data points, at  [towardsai.net/book](http://towardsai.net/book).

In [ ]:
import deeplake

# Connect to the training and testing datasets
ds = deeplake.load('hub://genai360/OpenOrca-1M-train-set')
ds_valid = deeplake.load('hub://genai360/OpenOrca-1M-valid-set')

print(ds)

    Dataset(path='hub://genai360/OpenOrca-1M-train-set',  
    read_only=True,  
    tensors=['id', 'question', 'response', 'system_prompt'])

The dataset features three key columns: question, the queries posed to the LLM; response, the model’s output or answers to these questions; and `system_prompt`, the initial instructions that set the context for the model, such as “you are a helpful assistant.”

For simplicity, this chapter focuses solely on the first two columns. However, incorporating system prompts into text formatting can be advantageous. The text is structured in the format `Question: xxx\n\nAnswer: yyy`, with the question and answer separated by two newline characters. You can also experiment with different formats, such as `System: xxx\n\nQuestion: yyy\n\nAnswer: zzz`, to include the system prompts from the dataset.

In [ ]:
def prepare_sample_text(example):
    """Prepare the text from a sample of the dataset."""
    text = f"""Question: {example['question'][0]}\n\nAnswer: {example['response'][0]}"""
    return text

Next, load the OPT model tokenizer:

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("facebook/opt-1.3b")

Use the `ConstantLengthDataset` class to aggregate data. This will maximize usage within the 2K input size constraint and improve training efficiency.

In [ ]:
from trl.trainer import ConstantLengthDataset

train_dataset = ConstantLengthDataset(
    tokenizer,
    ds,
    formatting_func=prepare_sample_text,
    infinite=True,
    seq_length=2048
)

eval_dataset = ConstantLengthDataset(
    tokenizer,
    ds_valid,
    formatting_func=prepare_sample_text,
    seq_length=1024
)

iterator = iter(train_dataset)
sample = next(iterator)
print(sample)

train_dataset.start_iteration = 0

    {'input_ids': tensor([ 16, 358, 828, ..., 137, 79, 362]),  
    'labels': tensor([ 16, 358, 828, ..., 137, 79, 362])}

### Initialize the Model and Trainer

The following code sets the LoRA configuration. The parameter r=16 sets the rank of the LoRA layers, controlling the reduction in dimensionality. The `lora_alpha=32` parameter adjusts the scaling factor for the adaptation, impacting how much the new parameters contribute to the model. `lora_dropout=0.05` specifies a dropout rate of 5%, adding regularization during training. The `bias="none"` parameter indicates no bias is added to the model. Finally, `task_type="CAUSAL_LM"` defines the task type as causal language modeling, which is typically used for autoregressive text generation models. We’ll later load the model in a quantized mode, effectively implementing QLoRA.

In [ ]:
from peft import LoraConfig

lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

Instantiate the `TrainingArguments`, which define the hyperparameters of the training process:


In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./OPT-fine_tuned-OpenOrca",
    dataloader_drop_last=True,
    evaluation_strategy="steps",
    save_strategy="steps",
    num_train_epochs=2,
    eval_steps=2000,
    save_steps=2000,
    logging_steps=1,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    learning_rate=1e-4,
    lr_scheduler_type="cosine",
    warmup_steps=100,
    gradient_accumulation_steps=1,
    bf16=True,
    weight_decay=0.05,
    ddp_find_unused_parameters=False,
    run_name="OPT-fine_tuned-OpenOrca",
    report_to="wandb",
)

Set a `BitsAndBytes` configuration. This new class package runs the quantization operation and loads the model in a 4-bit format. We will use the NF4 data type for weights and the nested quantization strategy to reduce memory usage while maintaining performance.

Next, specify that the training process computations be carried out in the `bfloat16` format.

The QLoRA method integrates LoRA with quantization to optimize memory usage further. To enable this functionality, include the `quantization_config` when initializing the model.

In [ ]:
import torch
from transformers import BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(
   load_in_4bit=True,
   bnb_4bit_quant_type="nf4",
   bnb_4bit_use_double_quant=True,
   bnb_4bit_compute_dtype=torch.bfloat16
)

Use the `AutoModelForCasualLM` class to load the OPT model’s pre-trained weights containing 1.3 billion parameters. Note that the model will be loaded using the GPUs registered in the system.

In [ ]:
from transformers import AutoModelForCausalLM
from accelerate import Accelerator

model = AutoModelForCausalLM.from_pretrained(
  "facebook/opt-1.3b",
    quantization_config=quantization_config,
    device_map={"": Accelerator().process_index}
)

Change the model architecture before initializing the trainer object to improve its efficiency. This requires casting specific layers of the model to complete precision (32 bits), including LayerNorms and the final language modeling head.

In [ ]:
from torch import nn

for param in model.parameters():
  param.requires_grad = False
  if param.ndim == 1:
    param.data = param.data.to(torch.float32)

model.gradient_checkpointing_enable()
model.enable_input_require_grads()

class CastOutputToFloat(nn.Sequential):
  def forward(self, x): return super().forward(x).to(torch.float32)
model.lm_head = CastOutputToFloat(model.lm_head)

The `SFTTrainer` class will begin training using the initialized dataset, the model, and the training arguments:

In [ ]:
from trl import SFTTrainer

trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    peft_config=lora_config,
    packing=True,
)

print("Training...")

In [ ]:
trainer.train()

The `SFTTrainer` instance will automatically establish checkpoints during the training process, as given by the `save_steps` argument (from `TrainingArguments`), and save them to the `./OPT-fine_tuned-OpenOrca` directory.

Merge the LoRA layers with the base model to form a standalone network. The following code will handle the merging process:

In [ ]:
from transformers import AutoModelForCausalLM
import torch

model = AutoModelForCausalLM.from_pretrained(
  "facebook/opt-1.3b", return_dict=True, torch_dtype=torch.bfloat16
)

from peft import PeftModel

# Load the Lora model
model = PeftModel.from_pretrained(model, "./OPT-fine_tuned-OpenOrca/<step>")
model.eval()

model = model.merge_and_unload()

model.save_pretrained("./OPT-fine_tuned-OpenOrca/merged")

The standalone model will be accessible on the .`/OPT-supervised_fine_tuned/merged` directory. This checkpoint will be used in the Reinforcement Learning section.

>💡[The Merged Model Checkpoint (2GB)](https://drive.google.com/file/d/1D9rH2kLiBgRR31xvelOcW09uHzrO5Fbv/view?usp=drive_link), [Weights & Bias Report](https://wandb.ai/ala_/GenAI360/runs/n6czwaqq?workspace=user-ala_), and the [fine-tuning requirements](https://github.com/towardsai/rag-ebook-files/blob/main/requirements-fine-tune.txt) are accessible at [towardsai.net/book](http://towardsai.net/book).

>(The provided requirements text file is a snapshot of all the packages on the server; not all of these packages are necessary for you)

## Training a Reward Model

• Find the  [Notebook](https://colab.research.google.com/github/towardsai/ragbook-notebooks/blob/main/notebooks/Chapter%2010%20-%20FineTuning_Reward_Model.ipynb)  for this section at  [towardsai.net/book](http://towardsai.net/book).

The reward model is designed to learn human preferences from labeled examples, guiding the LLM during the final stage of the RLHF process. It is exposed to examples of preferred and less desirable behaviors. It learns to mirror human preferences by assigning higher scores to preferred examples.

In essence, reward models perform a classification task, choosing the better option from a pair of sample interactions based on human feedback. Various network architectures can be used as reward models. A key consideration is whether the reward model should be similar to the base model to ensure it has adequate knowledge for practical guidance. However, smaller models such as DeBERTa or RoBERTa have also demonstrated efficiency. If resources permit, exploring larger models can be beneficial.

Install the essential libraries with the command `!pip install -q transformers==4.32.0 deeplake==3.6.19 sentencepiece==0.1.99 trl==0.6.0.`

### The Dataset

>💡Note that the datasets in this step contain inappropriate language and offensive words. This approach aligns the model’s behavior by instructing the model not to replicate it.

For the RLHF process, we use the “[helpfulness/harmless](https://github.com/anthropics/hh-rlhf)” dataset from Anthropic. This dataset is tailored for RLHF and offers an in-depth understanding of the approach. Find  [the study](https://arxiv.org/abs/2204.05862)  and the dataset at  [towardsai.net/book](http://towardsai.net/book).

The following code will set up the data loader objects for the training and validation sets:

In [ ]:
import deeplake

ds = deeplake.load('hub://genai360/Anthropic-hh-rlhf-train-set')
ds_valid = deeplake.load('hub://genai360/Anthropic-hh-rlhf-test-set')

print(ds)

Dataset(path='hub://genai360/Anthropic-hh-rlhf-train-set', read_only=True, tensors=['chosen', 'rejected'])

Before structuring the dataset for the Trainer class, load the pre-trained tokenizer for DeBERTa (the reward model). The code should be recognizable; the `AutoTokenizer` class will locate the suitable initializer class and utilize the `.from_pretrained()` method to load the pre-trained tokenizer.

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-v3-base")

PyTorch’s `Dataset` class prepares the dataset for various downstream tasks. A pair of inputs is required to train a reward model. The first item will represent the selected (favorable) conversation, while the second will represent a talk rejected by labelers. The reward model will allocate a higher score to the chosen sample and a lower score to the rejected samples.

The code below tokenizes the samples and combines them into a single Python dictionary:

In [ ]:
from torch.utils.data import Dataset

class MyDataset(Dataset):
    def __init__(self, dataset):
        self.dataset = dataset

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):

      chosen = self.dataset.chosen[idx].text()
      rejected = self.dataset.rejected[idx].text()

      tokenized_chosen = tokenizer(chosen, truncation=True, max_length=512, padding='max_length')
      tokenized_rejected = tokenizer(rejected, truncation=True, max_length=512, padding='max_length')

      formatted_input = {
        "input_ids_chosen": tokenized_chosen["input_ids"],
        "attention_mask_chosen": tokenized_chosen["attention_mask"],
        "input_ids_rejected": tokenized_rejected["input_ids"],
        "attention_mask_rejected": tokenized_rejected["attention_mask"],
      }

      return formatted_input

The `Trainer` class requires a dictionary with four keys for training. This includes the tokenized forms for chosen and rejected talks (`input_ids_chosen` and `input_ids_rejected`) and their respective attention masks `(attention_mask_chosen and attention_mask_rejected)`. Attention masks are necessary because they add padding tokens to standardize input sizes (up to the model’s maximum input size of 512 in this example), which warns the model that specific tokens at the end do not contain valuable information and can be ignored.

You can use the previously established class to construct an instance of the dataset or extract a single row from the dataset using the iter and next methods to validate the output keys and ensure that everything works as expected:

In [ ]:
train_dataset = MyDataset(ds)
eval_dataset = MyDataset(ds_valid)

# Print one sample row
iterator = iter(train_dataset)
one_sample = next(iterator)
print(list(one_sample.keys()))

    ['input_ids_chosen', 'attention_mask_chosen', 'input_ids_rejected',  
    'attention_mask_rejected']

### Initialize the Model and Trainer

Import the pre-trained DeBERTa model using the `AutoModelForSequenceClassification`. Set the number of labels (`num_labels`) to 1 since just a single score is needed to evaluate the quality of a sequence. A high score will signify content alignment, while a low score suggests the content may be unsuitable.

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(
    "microsoft/deberta-v3-base", num_labels=1
)

Create an instance of `TrainingArguments`, setting the intended hyperparameters. You can explore various hyperparameters based on the selection of pre-trained models and available resources. For example, if an out-of-memory error is encountered, a smaller batch size might be needed.

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="DeBERTa-reward-hh_rlhf",
    learning_rate=2e-5,
    per_device_train_batch_size=24,
    per_device_eval_batch_size=24,
    num_train_epochs=20,
    weight_decay=0.001,
    evaluation_strategy="steps",
    eval_steps=500,
    save_strategy="steps",
    save_steps=500,
    gradient_accumulation_steps=1,
    bf16=True,
    logging_strategy="steps",
    logging_steps=1,
    optim="adamw_hf",
    lr_scheduler_type="linear",
    ddp_find_unused_parameters=False,
    run_name="DeBERTa-reward-hh_rlhf",
    report_to="wandb",
)

The `RewardTrainer` class from the TRL library integrates all components, including the previously defined elements, such as the model, tokenizer, and dataset, and executes the training loop:

In [ ]:
from trl import RewardTrainer

trainer = RewardTrainer(
    model=model,
    tokenizer=tokenizer,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    max_length=512
)

trainer.train()

The `trainer` will automatically save the checkpoints, which will be used in the. Reinforcement Learning section.

💡[The Reward Model Checkpoint (Step 1000 - 2GB)](https://drive.google.com/file/d/1GWL-ayeeXDCMuqYjvDzgPJiRFmBY6X6Z/view?usp=drive_link), [Weights & Biases report](https://wandb.ai/ala_/GenAI360/runs/tqamj3nw?workspace=user-ala_), and [Requirements](https://github.com/towardsai/rag-ebook-files/blob/main/requirements-reward.txt) are accessible at [towardsai.net/book](http://towardsai.net/book).

(The provided requirements text file is a snapshot of all the packages on the server; not all of these packages are necessary for you)

## Reinforcement Learning

• Find the  [Notebook](https://colab.research.google.com/github/towardsai/ragbook-notebooks/blob/main/notebooks/Chapter%2010%20-%20FineTune_RLHF.ipynb)  for this section at  [towardsai.net/book](http://towardsai.net/book).

This final step in RLHF involves integrating the models we have developed earlier. At this point, the focus is on using the reward model trained earlier to align the fine-tuned model more closely with human feedback. During the training loop, a custom prompt will elicit a response from the fine-tuned OPT model. The reward model will then evaluate this response, assigning a score based on its resemblance to a response a human might generate.

In this reinforcement learning phase, safeguards ensure the model maintains the knowledge it has acquired and remains true to the original model’s foundational principles. The next step involves introducing the dataset, followed by an in-depth exploration of the process in the following subsections.

Install the necessary libraries with the command `!pip install -q transformers==4.32.0 accelerate==0.22.0 peft==0.5.0 trl==0.5.0 bitsandbytes==0.41.1 deeplake==3.6.19 wandb==0.15.8 sentencepiece==0.1.99`.

### The Dataset

There is considerable flexibility in selecting the dataset for this phase. The distinctive feature of this approach is that the reward model evaluates outputs independently of any specific labels, so the learning process doesn’t require a question-answer format.

We will use the OpenOrca dataset provided by Alpaca, a subset of the larger  [OpenOrca](https://huggingface.co/datasets/Open-Orca/OpenOrca)  dataset.

In [ ]:
import deeplake

# Connect to the training and testing datasets
ds = deeplake.load('hub://genai360/Alpaca-OrcaChat')

    Dataset(path='hub://genai360/Alpaca-OrcaChat', read_only=True,  
    tensors=['input', 'instruction', 'output'])

The dataset consists of three columns: `input`, the user’s prompt to the model; `instruction`, the directive for the model; and output, the model’s response. For the RL process, we will focus solely on the input column.

Before establishing a dataset class for appropriate formatting, load the pre-trained tokenizer corresponding to the fine-tuned model:

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("facebook/opt-1.3b", padding_side='left')

The trainer will need the query in its original and tokenized text formats in the following section. Therefore, the query will be retained as text, while the input_ids will signify the token IDs. Note that the query variable is a template for creating user prompts. This is structured in the format `Question: XXX\n\nAnswer:,` consistent with the one used during the SFT phase.

In [ ]:
from torch.utils.data import Dataset

class MyDataset(Dataset):
    def __init__(self, ds):
        self.ds = ds

    def __len__(self):
        return len(self.ds)

    def __getitem__(self, idx):

      query = "Question: " + self.ds.input[idx].text() + "\n\nAnswer: "
      tokenized_question = tokenizer(query, truncation=True,
max_length=400, padding='max_length', return_tensors="pt")

      formatted_input = {
        "query": query,
        "input_ids": tokenized_question["input_ids"][0],
      }

      return formatted_input

# Define the dataset object
myTrainingLoader = MyDataset(ds)


Create a collator function to convert individual samples from the data loader into data batches. This function will be provided to the `Trainer` class.

In [ ]:
def collator(data):
    return dict((key, [d[key] for d in data]) for key in data[0])